In [107]:
import numpy as np
import pandas as pd

In [108]:
asl_citizen_df=pd.read_csv('/kaggle/input/counting/ASL_Citizen_Gloss.csv')
asl_citizen_df.drop(['videos'],axis=1,inplace=True)
asl_citizen_df.rename(columns={'count': 'Count'}, inplace=True)
asl_citizen_df.head()

,Gloss,Count
0,shave,153
1,pineapple,152
2,erase,152
3,envelope,151
4,drown,150


In [109]:
msasl_df=pd.read_csv('/kaggle/input/counting/msasl_words_count.txt',sep=': ',names=['Gloss','Count'],engine='python')
msasl_df['Count']=msasl_df['Count'].astype(int)
msasl_df.head()

,Gloss,Count
0,eat,33
1,bird,31
2,teacher,31
3,finish,30
4,fish,30


In [110]:
wlasl_df=pd.read_csv('/kaggle/input/counting/wlasl_word_counts.txt',sep=': ',names=['Gloss','Count'],engine='python')
wlasl_df['Count']=wlasl_df['Count'].astype(int)
wlasl_df.head()

,Gloss,Count
0,before,16
1,cool,16
2,thin,16
3,drink,15
4,go,15


In [111]:
wlasl_df['source'] = 'wlasl'
msasl_df['source'] = 'msasl'
asl_citizen_df['source'] = 'asl_citizen'
asl_citizen_df.head()

,Gloss,Count,source
0,shave,153,asl_citizen
1,pineapple,152,asl_citizen
2,erase,152,asl_citizen
3,envelope,151,asl_citizen
4,drown,150,asl_citizen


In [112]:
import re

def clean_gloss(x):
    x = str(x).strip().lower()                 # remove spaces, lowercase
    # only remove trailing numbers if there is at least one letter
    if re.search('[a-zA-Z]', x):
        x = re.sub(r'\d+$', '', x)            # remove trailing digits
    return x

for df in [wlasl_df, msasl_df, asl_citizen_df]:
    df['Gloss'] = df['Gloss'].apply(clean_gloss)
asl_citizen_df.head()

,Gloss,Count,source
0,shave,153,asl_citizen
1,pineapple,152,asl_citizen
2,erase,152,asl_citizen
3,envelope,151,asl_citizen
4,drown,150,asl_citizen


In [113]:
all_df = pd.concat([wlasl_df, msasl_df, asl_citizen_df], ignore_index=True)
merged_counts = (
    all_df.groupby('Gloss').agg(
        Count=('Count', 'sum'),                     # sum counts
        source=('source', lambda x: ','.join(sorted(set(x))))  # unique sources joined
    ).reset_index()
)
merged_counts['Count']=merged_counts['Count'].astype(int)
merged_counts.head()

,Gloss,Count,source
0,100,2,msasl
1,1dollar,31,asl_citizen
2,21,9,msasl
3,22,9,msasl
4,23,7,msasl


In [114]:
merged_counts = merged_counts.sort_values(by='Count', ascending=False)  # descending order
merged_counts.head()

,Gloss,Count,source
924,envelope,160,"asl_citizen,msasl,wlasl"
1995,pineapple,159,"asl_citizen,msasl"
927,erase,157,"asl_citizen,wlasl"
2380,shave,157,"asl_citizen,wlasl"
795,dog,152,"asl_citizen,msasl,wlasl"


In [115]:
#verify 
merged_counts[merged_counts['Gloss'] == 'shave']

,Gloss,Count,source
2380,shave,157,"asl_citizen,wlasl"


In [116]:
asl_citizen_df[asl_citizen_df['Gloss']=='shave']

,Gloss,Count,source
0,shave,153,asl_citizen


In [117]:
wlasl_df[wlasl_df['Gloss']=='shave']

,Gloss,Count,source
1844,shave,4,wlasl


In [118]:
msasl_df[msasl_df['Gloss']=='shave']

,Gloss,Count,source


In [119]:
merged_counts.to_csv('asl_citizen_msasl_wsasl_gloss_count.csv',index=False)